In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [2]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFace, HuggingFaceModel
import json

/home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

In [4]:
huggingface_model = HuggingFaceModel(
    image_uri="084375588776.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-inference:1.0",
   model_data="s3://sagemaker-us-west-2-084375588776/model.tar.gz",
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.46.3", # transformers version used
   pytorch_version="2.5.1+cu121", # pytorch version used
   py_version="py310", # python version of the DLC
)

In [5]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.g5.xlarge",
   container_startup_health_check_timeout=300,
)

[03/10/25 00:00:30] INFO     Creating model with name:                                              ]8;id=756972;file:///home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=257089;file:///home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-inference-2025-03-10-00-00-30-617                                 

[03/10/25 00:00:31] INFO     Creating endpoint-config with name                                     ]8;id=137202;file:///home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=773917;file:///home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/session.py#5937\5937]8;;\
                             huggingface-pytorch-inference-2025-03-10-00-00-31-412                                 

                    INFO     Creating endpoint with name                                            ]8;id=820645;file:///home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=915245;file:///home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/session.py#4759\4759]8;;\
                             huggingface-pytorch-inference-2025-03-10-00-00-31-412                                 

-----------------------

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # deploy model to SageMaker Inference                                                        │
│ ❱ 2 predictor = huggingface_model.deploy(                                                        │
│   3    initial_instance_count=1,                                                                 │
│   4    instance_type="ml.g5.xlarge"                                                              │
│   5 )                                                                                            │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/huggingface/model. │
│ py:320 in deploy                                                                                 │
│                                                                                                  │
│   317 │   │   │   │   inference_tool=inference_tool,                                             │
│   318 │   │   │   )                                                                              │
│   319 │   │                                                                                      │
│ ❱ 320 │   │   return super(HuggingFaceModel, self).deploy(                                       │
│   321 │   │   │   initial_instance_count,                                                        │
│   322 │   │   │   instance_type,                                                                 │
│   323 │   │   │   serializer,                                                                    │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/model.py:1786 in   │
│ deploy                                                                                           │
│                                                                                                  │
│   1783 │   │   │   if is_explainer_enabled:                                                      │
│   1784 │   │   │   │   explainer_config_dict = explainer_config._to_request_dict()               │
│   1785 │   │   │                                                                                 │
│ ❱ 1786 │   │   │   self.sagemaker_session.endpoint_from_production_variants(                     │
│   1787 │   │   │   │   name=self.endpoint_name,                                                  │
│   1788 │   │   │   │   production_variants=[production_variant],                                 │
│   1789 │   │   │   │   tags=tags,                                                                │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/clean/lib/python3.11/site-packages/sagemaker/session.py:5940 in │
│ endpoint_from_production_variants                                                                │
│                                                                                                  │
│   5937 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   5938 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   5939 │   │                                                                                     │
│ ❱ 5940 │   │   return self.create_endpoint(                                                      │
│   5941 │   │   │   endpoint_name=name,                                                           │
│   5942 │   │   │   config_name=name,                       